In [1]:
from spacepy import pycdf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
from scipy import stats
from mpl_toolkits.mplot3d import axes3d
import matplotlib
import matplotlib.colors as colors
from collections import Counter
from scipy.optimize import curve_fit

In [2]:
cdf = pycdf.CDF('/Users/mayur/PhD/Cluster_data/CL_SP_AUX__20020111_000000_20020420_000000_V150526.cdf')

In [3]:
for (i, item) in enumerate(cdf, start=0):
    print(i, item, cdf[i])

0 time_tags__CL_SP_AUX CDF_EPOCH [142560]
1 half_interval__CL_SP_AUX CDF_FLOAT [] NRV
2 status__CL_SP_AUX CDF_INT4 [142560, 5]
3 sc_orbit_num__CL_SP_AUX CDF_FLOAT [142560]
4 sc_r_xyz_gse__CL_SP_AUX CDF_FLOAT [142560, 3]
5 sc_v_xyz_gse__CL_SP_AUX CDF_FLOAT [142560, 3]
6 sc_dr1_xyz_gse__CL_SP_AUX CDF_FLOAT [142560, 3]
7 sc_dr2_xyz_gse__CL_SP_AUX CDF_FLOAT [142560, 3]
8 sc_dr3_xyz_gse__CL_SP_AUX CDF_FLOAT [142560, 3]
9 sc_dr4_xyz_gse__CL_SP_AUX CDF_FLOAT [142560, 3]
10 sc_at1_lat__CL_SP_AUX CDF_FLOAT [142560]
11 sc_at1_long__CL_SP_AUX CDF_FLOAT [142560]
12 sc_at2_lat__CL_SP_AUX CDF_FLOAT [142560]
13 sc_at2_long__CL_SP_AUX CDF_FLOAT [142560]
14 sc_at3_lat__CL_SP_AUX CDF_FLOAT [142560]
15 sc_at3_long__CL_SP_AUX CDF_FLOAT [142560]
16 sc_at4_lat__CL_SP_AUX CDF_FLOAT [142560]
17 sc_at4_long__CL_SP_AUX CDF_FLOAT [142560]
18 sc_config_QG__CL_SP_AUX CDF_FLOAT [142560]
19 sc_config_QR__CL_SP_AUX CDF_FLOAT [142560]
20 sc_dr_min__CL_SP_AUX CDF_FLOAT [142560]
21 sc_dr_max__CL_SP_AUX CDF_FLOAT [142560

In [4]:
epoch = cdf[0][...]
position = cdf[4][...]

# Verifying Cluster is in the Pristine Solar Wind

#### _Foster, A.C. (Thesis, 2017), Chapter 6.1.2_ ##

Earth's bow shock position (Chao et al., 2002):

$$ r = r_{0}(\frac{1 + \epsilon}{1 + \epsilon \cos \theta})^{\alpha} $$

Dayside:

$$ \theta = \sin^{-1} (\frac{\sqrt{p_{y}^{2}+p_{z}^{2}}}{\sqrt{p_{x}^{2}+p_{y}^{2}+p_{z}^{2}}}) $$

Nightside:

$$ \theta = \pi - \sin^{-1} (\frac{\sqrt{p_{y}^{2}+p_{z}^{2}}}{\sqrt{p_{x}^{2}+p_{y}^{2}+p_{z}^{2}}}) $$

Spacecraft potition:

$$ d = \sqrt{p_{x}^{2}+p_{y}^{2}+p_{z}^{2}} $$

Only include $d > r + BS$

$BS = 4R_{E}$

In [127]:
epsilon = 1.029
Bz = -0.35
Dp = 2.48
beta = 2.08
Mms = 6.96
a1 = 11.1266
a3 = -0.0005
a4 = 2.5966
a5 = 0.8182
a6 = -0.017
a7 = -0.0122
a8 = 1.3007
a9 = -0.0049
a10 = -0.0328
a11 = 6.047
a14 = -0.002
a12 = 1.029
Re = 6371
BS = 4*Re #to ensure we are not magnetically connected to the bow shock 

alpha = a5*(1+a6*Bz)*(1+a7*Dp)*(1+a10*np.log(1+beta))*(1+a14*Mms)
r0 = a1*(1+a3*Bz)*(1+a9*beta)*(1+a4*(((a8-1)*(Mms**2) + 2)/((a8+1)*(Mms**2))))*Dp**(-1/a11)
r0 = r0*Re
print(alpha, r0/Re)

0.7580145569010709 13.137494179562479


In [128]:
d = np.sqrt(position[:,0]**2 + position[:,1]**2 + position[:,2]**2)

theta = []
for i in range(len(epoch)):
    if position[i,0] > 0:
        theta_i = np.arcsin((np.sqrt(position[i,1]**2 + position[i,2]**2))/(np.sqrt(position[i,0]**2 + position[i,1]**2 + position[i,2]**2)))
    else:
        theta_i = np.pi - np.arcsin((np.sqrt(position[i,1]**2 + position[i,2]**2))/(np.sqrt(position[i,0]**2 + position[i,1]**2 + position[i,2]**2)))
    theta.append(theta_i)

In [129]:
r = r0*(((1+epsilon)/(1+epsilon*np.cos(theta))))**alpha
limit = d-(r+BS)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in power
  """Entry point for launching an IPython kernel.


In [130]:
outside = []

for i in range(len(epoch)):
    if limit[i] > 0:
        pristine = 1
    else:
        pristine = 0
    outside.append(pristine)
    
print(Counter(outside))

start_times = []
end_times = []

for i in range(0,len(epoch)):
    if outside[i]>outside[i-1]:
        start_times.append(epoch[i])
    if outside[i]<outside[i-1]:
        end_times.append(epoch[i-1])

for (i, item) in enumerate(start_times, start=0):
    print(i, start_times[i], end_times[i])

Counter({0: 525478, 1: 122})
0 2018-02-28 03:32:30 2018-02-28 03:32:30
1 2018-03-02 09:12:30 2018-03-02 10:12:30
2 2018-03-04 15:22:30 2018-03-04 16:21:30


In [38]:
#Merka et al., 2003: Earth’s bow shock and magnetopause in the case of a field-aligned upstream flow
limit_test = (position[:,0]/Re) - (30-(3/250)*(((position[:,1]**2)+position[:,2]**2)/(Re**2))) 